In [1]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json
sys.path.insert(1, '../')

from utils.get_locations import HEALTH_FACILITY_TOWNS

In [2]:
data_dir = '/Users/abdulkadirgokce/Documents/MSFeCare/MSF-eCARE-filedrop'

!ls $data_dir

clean_data.csv                          metadata.json
cluster_points.csv                      metadata_patient.json
clustered_data.csv                      ml-dataset-datadictionary_20211001.xlsx
clustering_model.pkl                    model_kp.pkl
datadictionary-workinprogress.xlsx      patient_data.csv
full-310k - Copy xl.xlsx                processed_data.csv
full-310k - Copy.csv                    ~$full-310k - Copy xl.xlsx
gps_coordinates.json


In [3]:
filename = 'full-310k - Copy.csv'
file_dir = os.path.join(data_dir, filename)
df = pd.read_csv(file_dir)
df

/Users/abdulkadirgokce/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,2,5,8,9,43,46,48,51,54,114,129,130,131,134,135,136,154,156,158,160,182,183,184,186,187,191,202,203,204,208,212,234,248,252,265,266) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


source project   hf             created  slot_id a_age a_gender2  \
0           0   RCA-1    2  15feb2017 12:11:47    156.0  18.0      male   
1           0   RCA-1    3  01mar2017 08:59:36    169.0   NaN       NaN   
2           0   RCA-1    1  05jan2017 12:39:43    150.0  12.0      male   
3           0   RCA-1    3  16aug2017 10:05:44    186.0  12.0      male   
4           0   RCA-1    1  28apr2017 19:19:26    167.0  12.0      male   
...       ...     ...  ...                 ...      ...   ...       ...   
310299      8   RCA-2  801  12mar2020 09:55:48    158.0    37    female   
310300      8   RCA-2  801  08jan2020 08:26:21    158.0     5      male   
310301      8   RCA-2  801  04apr2020 10:06:07    158.0    26    female   
310302      8   RCA-2  801  16jul2020 14:42:21    158.0    17      male   
310303      8   RCA-2  801  30jun2020 09:18:21    158.0    12    female   

        a_weight                                 abdo_ds abdo_inguin_mass  \
0           10.2                                     NaN              NaN   
1            NaN                                     NaN              NaN   
2            7.0                                     NaN              NaN   
3            8.0                                     NaN              NaN   
4            9.8                                     NaN              NaN   
...          ...                                     ...              ...   
310299      12.6                                     NaN                0   
310300       7.2                                     NaN                0   
310301      12.1  [1fafe800-fd5b-11e3-a3ac-0800200c9a66]                0   
310302      10.0                                     NaN                0   
310303       7.9                                     NaN                0   

        ...  uti_vomiting  uvul_complication  uvulectomy vacci_card  \
0       ...           0.0                NaN         NaN        NaN   
1       ...           0.0                NaN         NaN        NaN   
2       ...           0.0                NaN         NaN        NaN   
3       ...           0.0                NaN         NaN        NaN   
4       ...           0.0                NaN         NaN        NaN   
...     ...           ...                ...         ...        ...   
310299  ...           0.0                0.0         0.0        0.0   
310300  ...           0.0                0.0         0.0        0.0   
310301  ...           0.0                0.0         0.0        0.0   
310302  ...           0.0                0.0         0.0        1.0   
310303  ...           0.0                0.0         0.0        0.0   

          vaccines  vsd_added_signs  wfa  wfa2  wfa_less_than_neg3sd  wfh2  
0              NaN              NaN  NaN   NaN                   NaN   NaN  
1              NaN              NaN  NaN   NaN                   NaN   NaN  
2              NaN              NaN  NaN   NaN                   NaN   NaN  
3              NaN              NaN  NaN   NaN                   NaN   NaN  
4              NaN              NaN  NaN   NaN                   NaN   NaN  
...            ...              ...  ...   ...                   ...   ...  
310299      notime              NaN  NaN   0.0                   0.0   NaN  
310300         NaN              NaN  NaN   0.0                   0.0   NaN  
310301      notime              NaN  NaN   0.0                   0.0   NaN  
310302  incomplete              NaN  NaN   0.0                   0.0   NaN  
310303      notime              NaN  NaN  -1.0                   0.0   NaN  

[310304 rows x 271 columns]

In [4]:
df.info(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310304 entries, 0 to 310303
Data columns (total 271 columns):
 #    Column                            Dtype  
---   ------                            -----  
 0    source                            object 
 1    project                           object 
 2    hf                                object 
 3    created                           object 
 4    slot_id                           float64
 5    a_age                             object 
 6    a_gender2                         object 
 7    a_weight                          float64
 8    abdo_ds                           object 
 9    abdo_inguin_mass                  object 
 10   abdo_intense_testispain           float64
 11   abdo_painkiller_response          float64
 12   adre_response                     float64
 13   agree_tx                          object 
 14   allergy_peni                      float64
 15   anaphyl_confirmed                 float64
 16   bloody_stool      

In [5]:
# Drop entries where too many features are not present

df = df.drop(df.index[df['source'].isin(['abdominal', 'eau', 'chaud', 'sro', 'coartem'])]) # 5 entries
df = df.drop(df.index[df['t_ab_recommended'].isna()]) # 8 entries
df = df.drop(df.index[df['a_gender2'].isna()]) # 10 entries
df = df.reset_index()

## Categorical features

In [6]:
"""
group neuro_any
--d_unconscious
--d_convuls_his
--d_convuls_now
--danger_sign_neck
--danger_sign_interaction
--danger_sign_tonus
--s_limp
"""

neuro_features = [
    'd_unconscious',
    'd_convuls_his',
    'd_convuls_now',
    'danger_sign_neck',
    'danger_sign_interaction',
    'danger_sign_tonus',
    's_limp'
]

neuro_any = df[neuro_features].any(axis='columns', skipna=True).astype(int).rename('neuro_any')


In [7]:
"""
group as muac_low
--s_muac_orange
--s_muac_red
"""

muac_low = ((df['s_muac']=='orange') | (df['s_muac']=='red')).astype(int).rename('muac_low')

In [8]:
"""
s_measles group ms_measles with measles_danger_signs
"""

df.loc[df.index[df['measles_danger_signs']=='[1fafe800-fd5b-11e3-a3ac-0800200c9a66]'], 'measles_danger_signs'] = None
df.loc[df.index[df['measles_danger_signs']=='["1fafe800-fd5b-11e3-a3ac-0800200c9a66"]'], 'measles_danger_signs'] = None

measles_features = ['ms_measles', 'measles_danger_signs']
s_measles = df[measles_features].any(axis='columns', skipna=True).astype(int).rename('s_measles')

In [9]:
"""
group as nonspecific
--s_none
--no_symptom_classifying_reported
--no_symptom_ecare_reported
"""

nonspecific_features = ['s_none', 'no_symptom_classifying_reported', 'no_symptom_ecare_reported']
nonspecific = df[nonspecific_features].any(axis='columns', skipna=True).astype(int).rename('nonspecific')

In [10]:
"""
group as resp_any
--cough_ds
--cough_indrawing
--cough_malaise_his
--cough_prolonged
--cough_wheez_rhinitis
--cough_wheeze_rash
--cough_wheezing
--cough_oxygen
--cough_rr
--high_respiratory_rate
--s_cough
--danger_sign_grunting
"""

# Replace bad entries with None
df.loc[df.index[df['cough_ds']=='[1fafe800-fd5b-11e3-a3ac-0800200c9a66]'], 'cough_ds'] = None
df.loc[df.index[df['cough_ds']=='["1fafe800-fd5b-11e3-a3ac-0800200c9a66"]'], 'cough_ds'] = None

resp_features = [
    'cough_ds',
    'cough_indrawing',
    'cough_malaise_his',
    'cough_prolonged',
    'cough_wheez_rhinitis',
    'cough_wheeze_rash',
    'cough_wheezing',
    'cough_oxygen',
    'cough_rr',
    'high_respiratory_rate',
    's_cough',
    'danger_sign_grunting',
]

resp_any = df[resp_features].any(axis='columns', skipna=True).astype(int).rename('resp_any')

In [11]:
"""
group below into abdo_any
--hydration_assessed
--hydration_eye
--hydration_neuro
--hydration_skin
--hydration_thirst
--diarrhoea_prolonged
--bloody_stool
--abdo_ds
--s_diarr
--s_abdopain
--s_vomit
--danger_sign_vomit_confirmed
--danger_sign_vomit_only
--danger_sign_vomiting
"""

# Replace bad entries with None
df.loc[df.index[df['hydration_thirst']=='574143ff-2257-438e-932f-291250d6c2cf'], 'hydration_thirst'] = None
df.loc[df.index[df['hydration_thirst']=='9f1e50be-ea44-45bf-baba-15e6079b2268'], 'hydration_thirst'] = None
df.loc[df.index[df['hydration_thirst']=='5ad025d5-b7fe-4fe6-ac2d-aa4c75914cb4'], 'hydration_thirst'] = None
df.loc[df.index[df['abdo_ds']=='[1fafe800-fd5b-11e3-a3ac-0800200c9a66]'], 'abdo_ds'] = None

df.loc[df.index[df['hydration_neuro']=='normal'], 'hydration_neuro'] = 0
df.loc[df.index[df['hydration_skin']=='normal'], 'hydration_skin'] = 0
df.loc[df.index[df['hydration_thirst']=='normal'], 'hydration_thirst'] = 0

abdo_features = [
    'hydration_assessed',
    'hydration_eye',
    'hydration_neuro',
    'hydration_skin',
    'hydration_thirst',
    'diarrhoea_prolonged',
    'bloody_stool',
    'abdo_ds',
    's_diarr',
    's_abdopain',
    's_vomit',
    'danger_sign_vomit_confirmed',
    'danger_sign_vomit_only',
    'danger_sign_vomiting'
]

abdo_any = df[abdo_features].any(axis='columns', skipna=True).astype(int).rename('abdo_any')

In [12]:
"""
group below into ear_any
--ear_pain_new
--ear_pain_prolonged
--ear_tender_swelling
--ear_pus
--s_earpain
"""

df.loc[df.index[df['ear_pus']=='earpus_none'], 'ear_pus'] = 0

ear_features = [
    'ear_pain_new',
    'ear_pain_prolonged',
    'ear_tender_swelling',
    'ear_pus',
    's_earpain'   
]

ear_any = df[ear_features].any(axis='columns', skipna=True).astype(int).rename('ear_any')

In [13]:
"""
eye_sympt (group with s_eyepb)
"""

df.loc[df.index[df['eye_sympt']=='[1fafe800-fd5b-11e3-a3ac-0800200c9a66]'], 'eye_sympt'] = None

eye_features = ['eye_sympt', 's_eyepb']
eye_sympt = df[eye_features].any(axis='columns', skipna=True).astype(int).rename('eye_sympt')

In [14]:
"""
group as skin_any
--s_skin
--skin_symptoms
"""

df.loc[df.index[df['skin_symptoms']=='[1fafe800-fd5b-11e3-a3ac-0800200c9a66]'], 'skin_symptoms'] = None

skin_features = ['s_skin', 'skin_symptoms']
skin_any = df[skin_features].any(axis='columns', skipna=True).astype(int).rename('skin_any')

In [15]:
"""
group urine_signs
--lab_urine_pos
--s_dysuria
--s_hematuria
"""

urine_features = ['lab_urine_pos', 's_dysuria', 's_hematuria']
urine_signs = df[urine_features].any(axis='columns', skipna=True).astype(int).rename('urine_signs')

In [16]:
"""
redo as hb_low if lab_hb is not <9
--lab_hb
"""
hb_low = (df['lab_hb']<9).astype(int).rename('hb_low')

In [17]:
df.loc[df.index[df['a_gender2']=='male'], 'a_gender2'] = 0
df.loc[df.index[df['a_gender2']=='female'], 'a_gender2'] = 1

df.loc[df.index[df['s_pallor']=='none'], 's_pallor'] = 0
df.loc[df.index[df['s_pallor']=='moderate'], 's_pallor'] = 1
df.loc[df.index[df['s_pallor']=='severe'], 's_pallor'] = 1

In [18]:
categorical_features = [
    #'a_gender2',
    'anaphyl_confirmed',
    'danger_sign_jaundice',
    'fever_prolonged',
    'lab_malaria_pos',
    'mouth_trush',
    's_oedema',
    's_pallor',
    's_drepano',
    # 's_fever_any' - Not present in dataset,
    's_throat',
    's_mouthpb',
    's_joint',
    'sam_u6add_ocp',
    'wfa_less_than_neg3sd'
]

grouped_features = [
    'neuro_any', 
    'muac_low', 
    's_measles', 
    'nonspecific', 
    'resp_any', 
    'abdo_any', 
    'ear_any', 
    'eye_sympt', 
    'skin_any', 
    'urine_signs', 
    'hb_low'
]

categorical_data = [
    neuro_any,
    muac_low,
    s_measles,
    nonspecific,
    resp_any,
    abdo_any,
    ear_any,
    eye_sympt,
    skin_any,
    urine_signs,
    hb_low,
    df[categorical_features].copy().fillna(0)
]

categorical_data = pd.concat(categorical_data, axis='columns')
categorical_features.extend(grouped_features)

## Spatial features

In [19]:
spatial_features = [
    'source',
    'project',
    'hf'
]


In [20]:
file_path = 'gps_coordinates.json'
if os.path.exists(file_path):
    with open(file_path, 'r') as file:
        gps_coordinates = json.load(file)
else:
    %run get_locations.py
    with open(file_path, 'r') as file:
        gps_coordinates = json.load(file)

gps_coordinates

{'Batangafo': [7.3039831, 18.2910914],
 'Kambakota': [7.1570168, 17.8721811],
 'Kabo': [7.6920799, 18.6243422],
 'Moyenne Sido': [8.2233649, 18.7159551],
 'Garin Wanzam': [13.6293456, 12.8959209],
 'Diffa': [13.3130749, 12.6136611],
 'Nigeria': [9.6000359, 7.9999721],
 'Nduta': [-3.6902255999999998, 30.776837487128315],
 'Hourara': [15.0644612, 0.6352474],
 'Boni': [15.0768761, -2.2211397],
 'Hombori': [15.2831164, -1.6990741],
 'Kidal': [19.802089000000002, 0.7253124568557916],
 'Abeibara': [19.3494891, 1.8443903],
 'Bobola Zangasso': [12.5364096, -4.9927581],
 'Konina': [12.4782163, -6.1795578],
 'Djitamana': [12.2465998, -5.636673],
 'Tchad': [15.6134137, 19.0156172],
 'Dagahaley': [0.19017905000000002, 40.28623989114301]}

In [21]:
df['hf'] = df['hf'].astype(int)

spatial_data = pd.DataFrame.from_dict(
    {
        'country': [project.split('-')[0] for project in df['project']],
        'hf_town': [HEALTH_FACILITY_TOWNS[hf] for hf in df['hf']],
        'latitude': [gps_coordinates[HEALTH_FACILITY_TOWNS[hf]][0] for hf in df['hf']],
        'longitude': [gps_coordinates[HEALTH_FACILITY_TOWNS[hf]][1] for hf in df['hf']]
    }
)

spatial_data

country    hf_town  latitude  longitude
0          RCA  Batangafo  7.303983  18.291091
1          RCA  Batangafo  7.303983  18.291091
2          RCA  Kambakota  7.157017  17.872181
3          RCA  Batangafo  7.303983  18.291091
4          RCA  Batangafo  7.303983  18.291091
...        ...        ...       ...        ...
310284     RCA       Kabo  7.692080  18.624342
310285     RCA       Kabo  7.692080  18.624342
310286     RCA       Kabo  7.692080  18.624342
310287     RCA       Kabo  7.692080  18.624342
310288     RCA       Kabo  7.692080  18.624342

[310289 rows x 4 columns]

## Temporal features

In [22]:
temporal_features = [
    'created'
]

In [23]:
datetimes = pd.to_datetime(df['created'], format='%d%b%Y %H:%M:%S')

temporal_data = pd.DataFrame()

In [24]:
temporal_data['week'] = [date.week for date in datetimes]
temporal_data['month'] = [date.month for date in datetimes]
temporal_data['year'] = [date.year for date in datetimes]
temporal_data['datetime'] = datetimes

temporal_data

week  month  year            datetime
0          7      2  2017 2017-02-15 12:11:47
1          1      1  2017 2017-01-05 12:39:43
2         33      8  2017 2017-08-16 10:05:44
3         17      4  2017 2017-04-28 19:19:26
4         48     12  2016 2016-12-02 10:22:03
...      ...    ...   ...                 ...
310284    11      3  2020 2020-03-12 09:55:48
310285     2      1  2020 2020-01-08 08:26:21
310286    14      4  2020 2020-04-04 10:06:07
310287    29      7  2020 2020-07-16 14:42:21
310288    27      6  2020 2020-06-30 09:18:21

[310289 rows x 4 columns]

In [25]:
# 53rd week does not exist
temporal_data.loc[temporal_data.index[(temporal_data['week']==53) &  (temporal_data['year']==2020)], 'week'] = 52
temporal_data.loc[temporal_data.index[(temporal_data['week']==53) &  (temporal_data['year']==2021)], 'week'] = 1

## Numerical features

In [26]:
numerical_features = [
    'a_age',
    #'a_weight'
]

#### Median Imputation
Instead of mean use median imputation to fill missing data since median is robust to outliers

In [27]:
df[numerical_features] = df[numerical_features].astype(float)
numerical_data = df[numerical_features].fillna(df[numerical_features].median())

#### Normalization

In [28]:
# Save statistics of features before normalization
normalization_stats = {}
normalization_stats['numerical_data_mean'] = numerical_data.mean().to_dict()
normalization_stats['numerical_data_std'] = numerical_data.std().to_dict()

normalization_stats

{'numerical_data_mean': {'a_age': 21.22278263167563},
 'numerical_data_std': {'a_age': 14.512732154169933}}

In [29]:
numerical_data = (numerical_data-numerical_data.mean())/numerical_data.std()

## Clean DataFrame

In [30]:
spatial_data.info(True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310289 entries, 0 to 310288
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   country    310289 non-null  object 
 1   hf_town    310289 non-null  object 
 2   latitude   310289 non-null  float64
 3   longitude  310289 non-null  float64
dtypes: float64(2), object(2)
memory usage: 9.5+ MB


In [31]:
temporal_data.info(True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310289 entries, 0 to 310288
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   week      310289 non-null  int64         
 1   month     310289 non-null  int64         
 2   year      310289 non-null  int64         
 3   datetime  310289 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 9.5 MB


In [32]:
numerical_data.info(True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310289 entries, 0 to 310288
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   a_age   310289 non-null  float64
dtypes: float64(1)
memory usage: 2.4 MB


In [33]:
categorical_data.info(True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310289 entries, 0 to 310288
Data columns (total 24 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   neuro_any             310289 non-null  int64  
 1   muac_low              310289 non-null  int64  
 2   s_measles             310289 non-null  int64  
 3   nonspecific           310289 non-null  int64  
 4   resp_any              310289 non-null  int64  
 5   abdo_any              310289 non-null  int64  
 6   ear_any               310289 non-null  int64  
 7   eye_sympt             310289 non-null  int64  
 8   skin_any              310289 non-null  int64  
 9   urine_signs           310289 non-null  int64  
 10  hb_low                310289 non-null  int64  
 11  anaphyl_confirmed     310289 non-null  float64
 12  danger_sign_jaundice  310289 non-null  float64
 13  fever_prolonged       310289 non-null  float64
 14  lab_malaria_pos       310289 non-null  float64
 15  

In [34]:
patient_data = pd.concat(
    [
        spatial_data, 
        temporal_data, 
        numerical_data, 
        categorical_data.astype(np.float64),
    ], 
    axis=1
)

patient_data = patient_data

In [35]:
patient_data.info(True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310289 entries, 0 to 310288
Data columns (total 33 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   country               310289 non-null  object        
 1   hf_town               310289 non-null  object        
 2   latitude              310289 non-null  float64       
 3   longitude             310289 non-null  float64       
 4   week                  310289 non-null  int64         
 5   month                 310289 non-null  int64         
 6   year                  310289 non-null  int64         
 7   datetime              310289 non-null  datetime64[ns]
 8   a_age                 310289 non-null  float64       
 9   neuro_any             310289 non-null  float64       
 10  muac_low              310289 non-null  float64       
 11  s_measles             310289 non-null  float64       
 12  nonspecific           310289 non-null  float64       
 13 

In [36]:
patient_data.to_csv(os.path.join(data_dir, 'patient_data.csv'), index=False)

In [37]:
metadata = {
    'spatial_features': list(spatial_data.columns),
    'temporal_features': list(temporal_data.columns),
    'numerical_features': list(numerical_data.columns),
    'categorical_features': list(categorical_data.columns),
    'normalization_stats' : normalization_stats
}

with open(os.path.join(data_dir, 'metadata_patient.json'), 'w') as file:
    json.dump(metadata, file, indent=4)

In [38]:
metadata

{'spatial_features': ['country', 'hf_town', 'latitude', 'longitude'],
 'temporal_features': ['week', 'month', 'year', 'datetime'],
 'numerical_features': ['a_age'],
 'categorical_features': ['neuro_any',
  'muac_low',
  's_measles',
  'nonspecific',
  'resp_any',
  'abdo_any',
  'ear_any',
  'eye_sympt',
  'skin_any',
  'urine_signs',
  'hb_low',
  'anaphyl_confirmed',
  'danger_sign_jaundice',
  'fever_prolonged',
  'lab_malaria_pos',
  'mouth_trush',
  's_oedema',
  's_pallor',
  's_drepano',
  's_throat',
  's_mouthpb',
  's_joint',
  'sam_u6add_ocp',
  'wfa_less_than_neg3sd'],
 'normalization_stats': {'numerical_data_mean': {'a_age': 21.22278263167563},
  'numerical_data_std': {'a_age': 14.512732154169933}}}